In [0]:
%run "/Databricks_MasterClass/Tutorial"

In [0]:
from pyspark.sql.functions import *

# Loading Json

In [0]:
dfjson = spark.read.format('json') \
            .option('inferSchema', 'true') \
                .option('header', 'true') \
                    .option('multiLine', 'true') \
                        .load('abfss://source@datalakemclassstorage.dfs.core.windows.net/Item.json')
display(dfjson)

In [0]:
dfjson.printSchema()

In [0]:
dffinal = dfjson.withColumn('New Batters', explode('batters.batter')) \
                .withColumn('New Topping', explode('topping')) \
                    .drop('batters', 'topping')

dffinal = dffinal.select('id', 'name', 'ppu', 'type', col('New Batters.id').alias('New Batters ID'), \
                                            col('New Batters.type').alias('New Batters Type'), \
                                            col('New Topping.id').alias('New Topping ID'), \
                                            col('New Topping.type').alias('New Topping Type')
                                            )
dffinal.display()

In [0]:
dfpay = spark.read.format('json') \
                .option('multiLine', True) \
                    .load('abfss://source@datalakemclassstorage.dfs.core.windows.net/Payload.json')

In [0]:
display(dfpay)

In [0]:
dfpay.withColumn('new geo', explode('geo.location')) \
    .withColumn('new items', explode('items')) \
        .withColumn('new geo city', col('new geo.city')) \
    .display()

In [0]:
dfpayfinal =dfpay.withColumn('new geo', explode('geo.location')) \
    .withColumn('new items', explode('items'))
    
dfpayfinal = dfpayfinal.select('User', 'device', col('new geo.city').alias('new geo city'), \
     col('new geo.state').alias('new geo state'), col('new geo.country').alias('new geo country'), \
         col('new items.description').alias('new items description'), col('new items.id').alias('new items id'), \
             col('new items.name').alias('new items name'))
dfpayfinal.display()
                 

# GIT HUB CI/CD Pipeline